# pyAstroTrader

## Download Data

This notebook contains the code to download the quotes from the ALPHAVANTAGE API for the asset indicated in the environment key: ASSET_TO_CALCULATE

The first step is to import all settings and also python modules that we are going to use.

In [ ]:
import os
import datetime
import pandas as pd
import numpy as np
import requests

from IPython.display import display, HTML

from settings import *

After all settings have been imported, we can check if the API key is correctly set in the environment variables of the process, and then use it to retrieve the quotes as a JSON file.

In [ ]:
if 'ALPHAVANTAGE_KEY' not in os.environ:
    raise ValueError('Need to define the API_KEY for the quotes')
    
ALPHAVANTAGE_KEY = os.environ['ALPHAVANTAGE_KEY']
URL='https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol={}&outputsize=full&apikey={}'.format(ASSET_TO_CALCULATE,ALPHAVANTAGE_KEY)

response = requests.get(URL)
if response.status_code != 200:
    raise ValueError('Error in getting the Data')
    
response = response.json()

After we receive the json file we need to retrieve the days that we need to process

In [ ]:
data_to_process = response['Time Series (Daily)']
days = list(data_to_process.keys())

With the days, we can now iterate over them and generate a list of dictionaries with the correct column name for the pandas dataframe to be used in our processing

In [ ]:
data_for_pandas = []
for current_day in days:
    current_day_date = datetime.datetime.strptime(current_day, '%Y-%m-%d')
    if current_day_date < DATE_MINIMAL:
        continue
    if ASSET_TO_CALCULATE in DATE_MINIMAL_STOCK:
        if current_day_date < DATE_MINIMAL_STOCK[ASSET_TO_CALCULATE]:
            continue
        
    if float(data_to_process[current_day]['4. close']) == 0:
        continue
        
    data_for_pandas.append(
        {
            'Date': current_day.replace('-',''),
            'Price': float(data_to_process[current_day]['4. close']),
            'Open':  float(data_to_process[current_day]['1. open']),
            'High':  float(data_to_process[current_day]['2. high']),
            'Low':  float(data_to_process[current_day]['3. low']),
            'Vol':  float(data_to_process[current_day]['6. volume'])
        }
    )
data_for_pandas = sorted(data_for_pandas, key=lambda x: x['Date'], reverse=False)

With the list of dictionaries, we can then create the pandas dataframe with the quotes data, using the constructor that takes as parameter the list of dictionaries

In [ ]:
df = pd.DataFrame(data_for_pandas)

We need to generate the CSV file that will be consumed by the ```CreateModel.ipynb``` notebook, we can use pandas builtin dataframe method to do it...

In [ ]:
output_csv_file='./input/{}_Daily.csv'.format(ASSET_TO_CALCULATE)
df.to_csv(output_csv_file)

And we use the excel method to generate a excel workbook with the quotes data...

In [ ]:
output_excel_file='./input/{}_Daily.xlsx'.format(ASSET_TO_CALCULATE)
df.to_excel(output_excel_file)